# Template

## Index
1. [Define required clock parameters](#Define-required-clock-parameters)
2. [Download necessary data](#Download-necessary-data)
3. [Load data](#-Load-data)
4. [Extract features and weights](#Extract-features-and-weights)
5. [Load weights into pyaging model](#Load-weights-into-pyaging-model)
6. [Add reference values](#Add-reference-values)
7. [Add preprocessing and postprocesssing steps](#Add-preprocessing-and-postprocesssing-steps)
8. [Check all data objects](#Check-all-data-objects)
9. [Write clock dictionary](#Write-clock-dictionary)
10. [Clear directory](#Clear-directory)

Let's first import some packages:

In [ ]:
import os
import marshal
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Define required clock parameters

Let's define some required information first:

In [ ]:
clock_name = 'example'
data_type = 'methylation'
model_class = 'LinearModel'
species = 'Homo sapiens'
year = 2022
approved_by_author = '⌛'
citation = 'Example citation'
doi = 'www.doi.example'
notes = None

## Download necessary data

#### Download directly with curl

In [ ]:
supplementary_url = "example_url.com"
supplementary_file_name = "example.xlsx"
os.system(f"curl -o {supplementary_file_name} {supplementary_url}")

#### Download GitHub repository

In [ ]:
github_url = "https://github.com/example/example_github.git"
github_folder_name = github_url.split('/')[-1].split('.')[0]
os.system(f"git clone {github_url}")

#### Download from R package

Let's first write an R script:

In [ ]:
%%writefile download.r

options(repos = c(CRAN = "https://cloud.r-project.org/"))
library(jsonlite)

install.packages("devtools")
devtools::install_github("example/example", build_vignettes = FALSE)
library(example)

write_json(example_object, "example.json", digits = 9)

Let's run the R script. For this, R needs to be installed.

In [ ]:
os.system("Rscript download.r")

## Load data

#### From Excel file

In [ ]:
df = pd.read_excel('example.xlsx', sheet_name='example_sheet', nrows=100)
reference_values_df = pd.read_excel('example.xlsx', sheet_name='example_sheet', nrows=100)
preprocessing_df = pd.read_excel('example.xlsx', sheet_name='example_sheet', nrows=100)

#### From CSV file

In [ ]:
df = pd.read_csv('example.csv', skiprows=0)
reference_values_df = pd.read_csv('example.csv', skiprows=0)
preprocessing_df = pd.read_csv('example.csv', skiprows=0)

#### From JSON file

In [ ]:
with open('example.json', 'r') as f:
    df_dict = json.load(f)[0]
with open('example.json', 'r') as f:
    reference_values_dict = json.load(f)[0]
with open('example.json', 'r') as f:
    preprocessing_df = json.load(f)[0]

## Extract features and weights

First, let's extract the features and weights:

In [ ]:
df['feature'] = df['example_probe_name']
df['coefficient'] = df['example_coefficient_name']

df.head()

Then, let's create lists for features and weights. Be careful about the intercept, as it usually shows up as a feature name.

In [ ]:
features = df['feature'][1:].tolist()
weights = torch.tensor(df['coefficient'][1:].tolist()).unsqueeze(0)
intercept = torch.tensor([df['coefficient'][0]]).float()

## Load weights into pyaging model

#### Linear model

In [ ]:
model = pya.models.LinearModel(input_dim=len(features))

model.linear.weight.data = weights.float()
model.linear.bias.data = intercept.float()

model

#### PC linear model

In [ ]:
model = pya.models.PCLinearModel(input_dim=len(features), pc_dim=num_components)

model.center.data = center.float()
model.rotation.data = rotation.float()
model.linear.weight.data = model_weights.float()
model.linear.bias.data = intercept.float()

## Add reference values

Some clocks have reference values in the case of missing features. It is also possible that these values are for preprocessing features rather than the clock features. Let's add a dictionary with the feature names as the keys.

In [ ]:
reference_feature_values = dict(zip(prepreprocessing_df['example'], reference_values_df['example']))

## Add preprocessing and postprocesssing steps

The preprocessing and postprocessing objects are dictionaries with the following format, with all items required. It takes in x in the form of a numpy array.

In [8]:
def preprocessing_function(x):
    y = preprocessing_helper_objects[0]
    return x + y
preprocessing_function_string = marshal.dumps(preprocessing_function.__code__)

preprocessing_helper_objects = [example]

preprocessing = {
    'name': 'add_example',
    'preprocessing_function': preprocessing_function_string,
    'preprocessing_helper_objects': preprocessing_helper_objects
}

Similarly is the case of postprocessing. Remember that your function must be compatible with torch and is applied to each number individually.

In [9]:
def postprocessing_function(x):
    y = postprocessing_helper_objects[0]
    return x + 1
    
postprocessing_function_string = marshal.dumps(postprocessing_function.__code__)

postprocessing_helper_objects = [1]

postprocessing = {
    'name': 'add_1',
    'postprocessing_function': postprocessing_function_string,
    'postprocessing_helper_objects': postprocessing_helper_objects
}

## Check all data objects

Let's print all data objects to check if they make sense.

#### features

In [ ]:
def my_print_function():
    print(f"There are {len(features)} features.")
    print(features)
pya.utils.print_to_scrollable_output(my_print_function)

#### reference_feature_values

In [ ]:
def my_print_function():
    if reference_feature_values:
        print(f"There are {len(reference_feature_values)} reference feature values.")
    print(reference_feature_values)
pya.utils.print_to_scrollable_output(my_print_function)

#### preprocessing

In [ ]:
def my_print_function():
    print(preprocessing)
    if preprocessing:
        print(preprocessing_helper_objects)
pya.utils.print_to_scrollable_output(my_print_function)

#### postprocessing

In [ ]:
def my_print_function():
    print(postprocessing)
    if postprocessing:
        print(postprocessing)
pya.utils.print_to_scrollable_output(my_print_function)

#### weight_dict

In [ ]:
def my_print_function():
    for name, param in model.named_parameters():
        print(f"Layer: {name}")
        print(f"Shape: {param.shape}")
        print(param.data)
pya.utils.print_to_scrollable_output(my_print_function)

## Write clock dictionary

Let's put everything together and save:

In [ ]:
clock_dict = {
    # Metadata
    'clock_name': clock_name,
    'data_type': data_type,
    'model_class': model_class,
    'species': species,
    'year': year,
    'approved_by_author': approved_by_author,
    'citation': citation,
    'doi': doi,
    "notes": notes,

    # Data
    'reference_feature_values': reference_feature_values if reference_feature_values else None,
    'preprocessing': preprocessing if preprocessing else None, 
    'features': features,
    'weight_dict': model.state_dict(),
    'postprocessing': postprocessing if postprocessing else None,
}

torch.save(clock_dict, f'../weights/{clock_name}.pt')

## Clear directory

Delete all files that are not clock jupyter notebooks:

In [ ]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)